In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wandb
from rdkit import Chem
from rdkit.Chem import AllChem
from torch.utils.data import DataLoader
from lightning.pytorch.loggers import WandbLogger
import pytorch_lightning as pl
import torch

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_hi_metrics
from mlp_model import MoleculeDataset, HiModel

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/hi/hiv/train_1.csv', index_col=0)
test = pd.read_csv('../../../../data/hi/hiv/test_1.csv', index_col=0)

train

,smiles,value
4,O=S(=O)(O)CCS(=O)(=O)O,0
21,CC(C)CCS(=O)(=O)O,0
90,O=S(=O)(O)CCO,0
106,O=S(=O)(O)CO,0
117,O=S(=O)(O)CCCCBr,0
...,...,...
40932,COC(=O)c1cc2cc3c(c(O)c2c(=O)o1)OC1(Oc2c(O)c4c(...,0
40973,CCCCC1C(OCOc2ccccc2)COC(=O)N1C(C)c1ccccc1,0
41024,CC(C)=CC1CC(C)C2CCC(C)C3C(=O)C(O)=C(C)C(=O)C123,0
41026,CCOC(=O)C12C(=O)C(C)CCC1C(C)CC2C=C(C)C,0


In [4]:
# assuming df is your DataFrame and 'value' is the column with the binary values
# separate majority and minority classes
df_majority = train[train.value==0]
df_minority = train[train.value==1]

# get counts of majority and minority class
count_majority = len(df_majority)
count_minority = len(df_minority)

# resample (oversample) minority class
df_minority_oversampled = df_minority.sample(count_majority, replace=True)

# combine majority class with oversampled minority class
df_oversampled = pd.concat([df_majority, df_minority_oversampled])

# to confirm if balanced
print(df_oversampled.value.value_counts())
train = df_oversampled

0    15097
1    15097
Name: value, dtype: int64


In [5]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]
train_fps = np.array(train_fps, dtype=np.float32)

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in test_mols]
test_fps = np.array(test_fps, dtype=np.float32)


[11:34:00] WARNING: not removing hydrogen atom without neighbors
[11:34:00] WARNING: not removing hydrogen atom without neighbors


In [6]:
train_dataset = MoleculeDataset(train_fps, np.array(train['value'], dtype=np.float32))
test_dataset = MoleculeDataset(test_fps, np.array(test['value'], dtype=np.float32))

In [7]:
train_dataset[0]

(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 0.0)

In [8]:
params = {
    'layers': [1024, 16],
    'use_dropout': False,
    'dropout': 0.0,
    'lr': 0.001,
    'l2': 0.002,
    'batch_size': 32,
}

In [9]:
param_dict = {
    'layers': [
        [1024, 32, 32],
        [1024, 16, 16],
        [1024, 32],
        [1024, 8, 4],
        [1024, 4]
    ],
    'use_dropout': [True],
    'dropout': [0.0, 0.0, 0.2, 0.4, 0.6],
    'lr': [0.01],
    'l2': [0.0, 0.0, 0.001, 0.005, 0.01],
    'batch_size': [32]
}

In [10]:
from sklearn.model_selection import ParameterSampler
from pytorch_lightning.callbacks import EarlyStopping

hyperparam_list = list(ParameterSampler(param_dict, n_iter=30))

best_score = 0.0
best_hyperparam = None

for i, hyperparam in enumerate(hyperparam_list):
    early_stop_callback = EarlyStopping(
        monitor='test_prc_auc',  # the metric to monitor
        min_delta=0.00,  # minimum change in the monitored quantity to qualify as an improvement
        patience=20,  # number of epochs with no improvement after which training will be stopped
        verbose=False,  # verbosity mode
        mode='max'  # 'min' for minimizing the metric, 'max' for maximizing
    )

    print('HYPERPARAMETER', i)
    print(hyperparam)

    train_dataloader = DataLoader(train_dataset, batch_size=hyperparam['batch_size'], shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=hyperparam['batch_size'], shuffle=False)

    model = HiModel(hyperparam)
    wandb.init(project='hiv_hi_ecfp4')
    wandb_logger = WandbLogger(project='hiv_hi_ecfp4')
    trainer = pl.Trainer(max_epochs=150, logger=wandb_logger, accelerator="gpu", devices=[1], default_root_dir=None, callbacks=[early_stop_callback])
    trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)
    score = early_stop_callback.best_score.detach().cpu().numpy()
    if score > best_score:
        best_score = score
        best_hyperparam = hyperparam

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


HYPERPARAMETER 0
{'use_dropout': True, 'lr': 0.01, 'layers': [1024, 8, 4], 'l2': 0.0, 'dropout': 0.6, 'batch_size': 32}
Sequential(
  (0): Linear(in_features=1024, out_features=8, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.6, inplace=False)
  (3): Linear(in_features=8, out_features=4, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.6, inplace=False)
  (6): Linear(in_features=4, out_features=1, bias=True)
  (7): Sigmoid()
)


wandb: Currently logged in as: steshinss. Use `wandb login --relogin` to force relogin


/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 8.2 K 
-------------------------------------
8.2 K     Trainable params
0         Non-trainable params
8.2 K     Total params
0.033     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [11]:
print(best_score)

0.21176462


In [12]:
print(best_hyperparam)

{'use_dropout': True, 'lr': 0.01, 'layers': [1024, 8, 4], 'l2': 0.01, 'dropout': 0.4, 'batch_size': 32}


In [13]:
best_hyperparam = {
    'use_dropout': True,
    'lr': 0.01,
    'layers': [1024, 8, 4],
    'l2': 0.01,
    'dropout': 0.4,
    'batch_size': 32
}

In [14]:
from pytorch_lightning.callbacks import EarlyStopping
early_stop_callback = EarlyStopping(
    monitor='test_prc_auc',  # the metric to monitor
    min_delta=0.00,  # minimum change in the monitored quantity to qualify as an improvement
    patience=20,  # number of epochs with no improvement after which training will be stopped
    verbose=False,  # verbosity mode
    mode='max'  # 'min' for minimizing the metric, 'max' for maximizing
)
train_dataloader = DataLoader(train_dataset, batch_size=best_hyperparam['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=best_hyperparam['batch_size'], shuffle=False)

model = HiModel(best_hyperparam)
wandb.init(project='hiv_hi_ecfp4')
wandb_logger = WandbLogger(project='hiv_hi_ecfp4')
trainer = pl.Trainer(max_epochs=150, logger=wandb_logger, accelerator="gpu", devices=[1], default_root_dir=None, callbacks=[early_stop_callback])
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)

Sequential(
  (0): Linear(in_features=1024, out_features=8, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.4, inplace=False)
  (3): Linear(in_features=8, out_features=4, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.4, inplace=False)
  (6): Linear(in_features=4, out_features=1, bias=True)
  (7): Sigmoid()
)


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
test_bedroc,▂▄▅▅▄▂▂▃▂▁▂▂▃▄▃▃▆▁▁▃▃▅▂▃█▃▆▃▄▆▁▂▂▂▄▁▁▅▇▂
test_prc_auc,▂▅▄▄▄▂▂▃▁▂▁▂▂▄▃▂▆▁▂▃▂▄▂▃█▃▆▃▄▅▁▂▂▂▄▂▂▄▆▁
test_roc_auc,▃▇▅▇▆▂▄▆▁▃▂▅▅▆▅▅█▂▄▆▅▆▅▆█▅█▆▅█▃▄▄▃▇▄▄▆▇▂
train_bedroc,▆▄█▂▅▅▃▅▆▅▇▇▇▄▂▆▇█▂▅▅▃▁▄▂▁▆▅▅▄▄▆▁▄▇▇▅▆▃▂
train_loss,▄▅▆▄▄▄▅▇▃▃▄▃▃▃█▄▃▄▅▇▇▄▄▁▃▅▆▄▂▁▄▅▆▂▂▅▂▄▃▆
train_prc_auc,▂▅█▆█▅▅▆▅▅▆▇▇▆▅▆▄▁▃▃▁▃▁▃▄▁▂▃▄▃▁▁▂▂▂▂▄▄▃▃
train_roc_auc,▃▆███▅▆▆▅▆▆▆▇▆▆▆▄▁▄▃▁▄▂▃▄▂▁▄▃▃▃▁▃▂▂▂▄▄▃▄
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,▁▇▃▅▂▅▁▄▂▃▃▁▄▃▂▃▄█▄▃█▄▄▄▅▃▃▄▆▃▄▃▃▃▃▃▅▆▅█
epoch,51


/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 8.2 K 
-------------------------------------
8.2 K     Trainable params
0         Non-trainable params
8.2 K     Total params
0.033     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [15]:
all_preds = []
all_true = []
with torch.no_grad():
    for _, batch in enumerate(test_dataloader):
        X, y = batch
        y_pred = model(X)
        all_preds.append(y_pred)
        all_true.append(y)

In [16]:
all_preds = np.squeeze(torch.cat(all_preds).cpu().numpy())
all_true = np.squeeze(torch.cat(all_true).cpu().numpy())

In [17]:
get_hi_metrics(pd.DataFrame({'value': all_true}), all_preds)

{'roc_auc': 0.6361899482052046,
 'bedroc': 0.10347608971049027,
 'prc_auc': 0.07209298842287899}

# Final Evaluation

In [20]:
def fit_predict(train, test):
    df_majority = train[train.value==0]
    df_minority = train[train.value==1]

    # get counts of majority and minority class
    count_majority = len(df_majority)
    count_minority = len(df_minority)

    # resample (oversample) minority class
    df_minority_oversampled = df_minority.sample(count_majority, replace=True)

    # combine majority class with oversampled minority class
    df_oversampled = pd.concat([df_majority, df_minority_oversampled])

    # to confirm if balanced
    train = df_oversampled

    train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
    train_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]
    train_fps = np.array(train_fps, dtype=np.float32)

    test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
    test_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in test_mols]
    test_fps = np.array(test_fps, dtype=np.float32)

    train_dataset = MoleculeDataset(train_fps, np.array(train['value'], dtype=np.float32))
    test_dataset = MoleculeDataset(test_fps, np.array(test['value'], dtype=np.float32))

    train_dataloader = DataLoader(train_dataset, batch_size=best_hyperparam['batch_size'], shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=best_hyperparam['batch_size'], shuffle=False)

    model = HiModel(best_hyperparam)
    trainer = pl.Trainer(max_epochs=10, accelerator="gpu", devices=1, default_root_dir=None, num_sanity_val_steps=0)
    trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)

    train_result = train.copy()
    train_predict_dataloader = DataLoader(train_dataset, batch_size=best_hyperparam['batch_size'], shuffle=False)
    all_preds = []
    with torch.no_grad():
        for _, batch in enumerate(train_predict_dataloader):
            X, _ = batch
            y_pred = model(X)
            all_preds.append(y_pred)
    all_preds = np.squeeze(torch.cat(all_preds).cpu().numpy())
    train_result['preds'] = all_preds

    test_result = test.copy()
    all_preds = []
    with torch.no_grad():
        for _, batch in enumerate(test_dataloader):
            X, _ = batch
            y_pred = model(X)
            all_preds.append(y_pred)
    all_preds = np.squeeze(torch.cat(all_preds).cpu().numpy())
    test_result['preds'] = all_preds

    return train_result, test_result

In [21]:
for i in [1, 2, 3]:
    train = pd.read_csv(f'../../../../data/hi/hiv/train_{i}.csv')
    test = pd.read_csv(f'../../../../data/hi/hiv/test_{i}.csv')

    train_preds, test_preds = fit_predict(train, test)
    train_preds.to_csv(f'../../../../predictions/hi/hiv/mlp_ecfp4/train_{i}.csv')
    test_preds.to_csv(f'../../../../predictions/hi/hiv/mlp_ecfp4/test_{i}.csv')


[12:33:13] WARNING: not removing hydrogen atom without neighbors
[12:33:13] WARNING: not removing hydrogen atom without neighbors
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Sequential(
  (0): Linear(in_features=1024, out_features=8, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.4, inplace=False)
  (3): Linear(in_features=8, out_features=4, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.4, inplace=False)
  (6): Linear(in_features=4, out_features=1, bias=True)
  (7): Sigmoid()
)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 8.2 K 
-------------------------------------
8.2 K     Trainable params
0         Non-trainable params
8.2 K     Total params
0.033     Total estimated model params size (MB)
/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing 

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
